# DMS-MaPseq

In [7]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../../src')
from util import *
from config import *
import ipynbname

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### # Mutations per read
- histogram
- x-axis = # mutations
- y-axis = # reads


In [8]:

savefig2(ipynbname.path(), '[A] Mutations per read')

### Mutation identity at each position
- stacked bar graph (ACUG coloring)
- x-axis = position (number/base/both??)
- y-axis = # reads


In [ ]:
savefig2(ipynbname.path(), '[B] Mutation identity at each position')

### Mutation fraction at each position 
- bar graph (ACUG coloring)
- x-axis = position (number/base/both??)
- y-axis = # reads


In [ ]:
savefig2(ipynbname.path(), '[C] Mutation fraction at each position')

### Read coverage per position
- bar graph
- x-axis = position (number/base/both??)
- y-axis = coverage fraction

In [ ]:
savefig2(ipynbname.path(), '[D] Read coverage per position')